# Introduction  

The goal of this assignment is to guide you through the solution of a puzzle using python.

The method you will be implementing is called backtracking ("tilbakesporing" in Norwegian). It is not strictly speaking an algorithm, since the details in the implementation varies a lot from problem to problem.

We will first be working on a very simple problem.

First a few comments:

- Note that exhaustive search is a viable approach to solving these problems, but we are not interested in such solutions. We want you to learn how to implement backtracking. Backtracking is something we use when exhaustive search is not viable.

- Although it might seem that we are solving silly puzzles for fun, the methods here are strong and are applicable to serious real world problems. Many real world problems can be thought of as solving a logical puzzle. If you are interested look up "constraint satisfaction problems" on the internet.

- We have divided up the assignment in a part we want everyone to do, and a part which should be thought of as one of the challenge assignments. 

We start with a quick review of some python that you could use. Most of this should be quite familiar by now. We don't say that you have to use this.

You should not import any other external libraries for this assignment. We will however need deepcopy from copy.



In [43]:
#
# Example code. Make sure you understand this code before you continue.
#

#
# Sets could be useful. Up to you though. I have not used it below.
#

# We create a set consisting of the numbers 0..9
mySet = {i for i in range(10)}

# We create a list of sets, each containing the numbers 0..9
mySets = [{i for i in range(10)} for j in range(5)]

# Some operations on sets
s1 = {1,2,3,4,5}
s2 = {3,4,5,6,7}

print(s1 & s2)  # intersection of two sets
print(s1 | s2)  # union of two sets
print(s1 ^ s2)  # symmetric difference (xor) of two sets
print(s1 - s2)  # difference of two sets

# adding an removing elements from sets
s1.remove(1) 
print(s1)
s1.add(10)
print(s1)

# We can compare sets
s1 = {1,2,3}
s2 = {1,2,3}
s3 = {3,4,5}
print(s1 == s2)
print(s2 == s3)


#
# A stack is a list where you only allow yourself to add and remove elements from the back of the list.
# The effect is that when we remove something it will always be the last element we added. This is called 
# last in first out or LIFO for short.
#
# We can add and remove elements using the .append and .pop methods
#
# Here is code for adding 1,2,3 to a list and then removing them in the order 3,2,1
#
myStack = []   
myStack.append(1)
print(myStack)
myStack.append(2)
print(myStack)
myStack.append(3)
print(myStack)
print(myStack.pop(), myStack)
print(myStack.pop(), myStack)
print(myStack.pop(), myStack)

#
# deepcopy will likely be useful. Here is a recap of the difference between copy and deepcopy.
#
from copy import deepcopy
myList = [[1],[2],[3]]
myList2 = myList.copy()
myList[0][0]=2   # changes myList2
print(myList2)
myList2 = deepcopy(myList)
myList[0][0]=10  # does not change myList2
print(myList2)


{3, 4, 5}
{1, 2, 3, 4, 5, 6, 7}
{1, 2, 6, 7}
{1, 2}
{2, 3, 4, 5}
{2, 3, 4, 5, 10}
True
False
[1]
[1, 2]
[1, 2, 3]
3 [1, 2]
2 [1]
1 []
[[2], [2], [3]]
[[2], [2], [3]]


# A simple problem

We are going to solve a simple problem. We are going to find all triples of numbers $(a,b,c)$ with $a,b,c=1..4$ so that $a<b<c$. Theses properties are called the "CONSTRAINTS" of the problem. A valid solution has to satisfy the constraints.

We start by defining our problem space and adding it to a list of partial solutions. (line 1 and 2). 

We then look at the last partial solution we created. We make an assumption (which could be wrong) about what the first number should be. We divide the problem into two parts, one where our assumption is correct and one where our assumption is incorrect. We have created "BRANCHES" in our problem.

Having made our assumption, we make a computation where we look at the consequences of our assumptions. This step is called "PROPAGATION".

Propagation can lead to invalid solutions, which are then discarded.

Backtracking with branching but without propagation is equivalent to brute search. 

Propagation without branching is sometimes possible, but often leads to extremely complicated code

The Backtracking Method is all about striking a balance between branching a lot (simple code, but slow) and propagating (complicated, but fast)

In this first example we do not do propagation - only branching. We have to run the loop 63 times to find all the solutions.




In [11]:
from copy import deepcopy

# setting up the problem
space = [[1,2,3,4] for i in range(3)]
partials = []
partials.append(space)

numRuns = 0

while partials != []:
    #
    # We obtain a partial solution from our list of partial solutions
    #
    current = partials.pop()
    
    #
    # Branching rule: we find the first index with a set with more than one element 
    #
    index = 0
    found = False
    for sol in current:
        if len(sol) > 1:
            found = True 
            break
        index += 1
    
    #
    # if an index was not found, we only have one element sets, and we cannot branch.
    # we check and see if we now have a solution, otherwise we skip forward
    #
    if not found:
        valid = True
        for i in range(len(current)-1):
            valid = valid and (current[i][0] < current[i+1][0])
        if valid: print(current)
        continue

    #
    # we branch at the index where the set is not one element.
    #
    val = current[index].pop()  # we remove one element from the current partial solution as one branch
    partials.append(current)    # we do not propagate in this example
    branch = deepcopy(current)  # we create the other branch
    branch[index] = [val]   
    partials.append(branch)     # we do not propagate in this example

    numRuns += 1


print(numRuns)

[[2], [3], [4]]
[[1], [3], [4]]
[[1], [2], [4]]
[[1], [2], [3]]
63


We add a propagation rule to make the algorithm more efficient. We only propagate one of the branches.

We get more complex code (slower) but have a lot shorter search (faster). Backpropagation is always a delicate balance between branching and propagation rules to get a program which runs fast. This problem however is too small to show any meaningful difference between the two.

But we now only have to run the loop 12 times to find all the solutions (compared to 63 above).


In [12]:
from copy import deepcopy

# setting up the problem
space = [[1,2,3,4] for i in range(3)]
partials = []
partials.append(space)

numRuns = 0

while partials != []:
    #
    # We obtain a partial solution from our list of partial solutions
    #
    current = partials.pop()
    
    #
    # Branching rule: we find the first index with a set with more than one element 
    #
    index = 0
    found = False
    for sol in current:
        if len(sol) > 1:
            found = True 
            break
        index += 1
    
    #
    # if an index was not found, we only have one element sets, and we cannot branch.
    # we check and see if we now have a solution, otherwise we skip forward
    #
    if not found:
        valid = True
        for i in range(len(current)-1):
            valid = valid and (current[i][0] < current[i+1][0])
        if valid: print(current)
        continue

    #
    # we branch at the index where the set is not one element.
    #
    val = current[index].pop()  # we remove one element from the current partial solution as one branch
    partials.append(current)    # we do not propagate in this example
    branch = deepcopy(current)  # we create the other branch
    branch[index] = [val]   

    # we propagate one branch
    for i in range(index):  # if 
        branch[i] = list(filter(lambda x : x < val, branch[i]))
    for i in range(index + 1, len(branch)):
        branch[i] = list(filter(lambda x : x > val, branch[i]))

    # we check for contradictions
    valid = True
    for s in branch:
        valid = valid and (s != [])
    
    # we add the branch to the stack
    if valid: partials.append(branch)     # we do not propagate in this example

    numRuns += 1


print(numRuns)

[[2], [3], [4]]
[[1], [3], [4]]
[[1], [2], [4]]
[[1], [2], [3]]
12


# Problem 1.

We have tiles with labels A, B, C, D, E.

We have 3 tiles of each of the tiles labelled A, B, C and D. We 4 tiles of the tile labelled E. All together we have 16 tiles. These are constraints for a solution.

The tiles should be placed on a 4 x 4 grid. 

For each position on the 4x4 grid there are neighbouring positions up, down, left and right, and the 4 diagonal directions up-left, up-right, down-left, down-right. So there are 8, 5 (on edge) or 3 (on corners) neighbours, depending on where on the grid we are located.

When placing the tiles, the tile in each position should have be different label from the labels on the tiles in each of the neighbouring positions. In other words, if two tiles are neighbours, their labels have to be different. This is the constraints that a solution has to satisfy.

To summarise: The constraints are

- there are 3 tiles labelled A
- there are 3 tiles labelled B
- there are 3 tiles labelled C
- there are 3 tiles labelled D
- there are 4 tiles labelled E
- if two tiles are neighbours, then they have a different label.

Find all possible placement of the tiles which satisfies the constraints. How many solutions are there?

Implement the solution to this problem using backtracking with propagation. You should use the code above as your basis for the solution. You will need to use some kind of propagation to get a full score.

Comment in the code exactly what your branching and propagation rules are.



In [ ]:
#
# Solution
#

# Challenge assignment (20 points)

This is a challenge assignment, but it does not require any complicated syntax from numpy or pyglet or similar. It is a mathematical problem which can be solved with simple programming concepts like loops and lists. This does not mean the problem is easy.

It is important that you have solved the previous problem before you start.

We want to study the previous puzzle for arbitrary grid size and arbitrary number of tiles for each label. We are interested in how many solutions there are.

So we have a collection of tiles labelled $1, 2, ....$ which we will place on a $a\times a$ grid.
The number of tiles with label $1$ is $n_1$, the number of tiles with label $2$ is $n_2$, the number of tiles with label $3$ is $n_3$, and so on. When we place tiles on the grid, neighbours should have different labels. To summarise, a possible solution is a selection of numbers $n_i$ and a placement of tiles on a grid, where the constraint is that:

- $n_i \leq n_j$ if $i<j$.
- there are $n_i$ tiles with label $i$.
- $\sum_i n_i = a^2$ (so that we can fill in the grid with tiles).
- if two tiles are neighbours on the grid, then they have different labels

So for instance, when $a=2$, we need to solve the problem number of tiles with labels corresponding to the sums $1+1+1+1, 1+1+2, 1+3, 2+2$ and $4$. 

For $a=3$, we need to consider $30$ different label counts, corresponding to different ways to write $3\cdot 3 = 9$ as a sum of positive integers. $9, 1+8, 2+7, 1+1+7$ are four of these possible $30$. For $a=4$, there are $231$ different ways to write $4^2=16$ as a sum of positive integers. For $a=5$ there are $1958$ and so on.

The printout from the program should be something like

for a = 2, and 4 = 4, there are 0 solutions,

for a = 2, and 1 + 3 = 4, there are 0 solutions,

for a = 2, and 2 + 2 = 4, there are 2 solutions, 

for a = 2, and 1 + 1 + 2 = 4, there are 4 solutions

for a = 2, and 1 + 1 + 1 + 1 = 4, there are 24 solutions

for a = 3, and 9 = 9, there are 0 solutions

for a = 3, and 1 + 8 = 9, there are ....  (and so on) 

....

For full score the program should be able to continue to run and give more answers for higher and higher $a$.

